## pybioviz test dashboards

In [1]:
import os, sys, io, random
import string
import numpy as np
import pandas as pd
from importlib import reload

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from IPython.display import HTML

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, CustomJS, Slider, HoverTool, NumeralTickFormatter, Arrow, NormalHead
from bokeh.models import LinearAxis, Range1d, DataRange1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot, column
import panel as pn
import panel.widgets as pnw
pn.extension()

from pybioviz import dashboards, utils, plotters

## random colors plot

In [6]:
def test_app():
    """Test dashboard"""
    
    def refresh(event):
        plot1.object = plotters.test1(cols=col_sl.value,rows=row_sl.value)
        plot2.object = plotters.test2(rows=row_sl.value)
        return
    
    title = pn.pane.Markdown('## pybioviz test plots')
    plot1 = pn.pane.Bokeh()    
    plot2 = pn.pane.Bokeh()
    col_sl = pnw.IntSlider(name='cols',value=30,start=5,end=200,step=1)
    col_sl.param.watch(refresh, 'value')
    row_sl = pnw.IntSlider(name='rows',value=10,start=5,end=100,step=1)
    row_sl.param.watch(refresh, 'value')
    col_sl.param.trigger('value')    
    app = pn.Column(title,col_sl,row_sl,plot1,plot2)
    return app

app = test_app()
app

Column
    [0] Markdown(str)
    [1] IntSlider(end=200, name='cols', start=5, value=30)
    [2] IntSlider(end=100, name='rows', start=5, value=10)
    [3] Bokeh(figure, align='auto', height=300, width=800)
    [4] Bokeh(figure, align='auto', height=200, width=1000)

## seq aligner test

In [3]:
m = pn.pane.Markdown("""## pybioviz test""")

aln_file = os.path.join(utils.datadir, 'test.aln')
aln = AlignIO.read(aln_file,'clustal')
seqview = plotters.plot_sequence_alignment(aln)
main = pn.Column(m,seqview)
main

Column
    [0] Markdown(str)
    [1] Bokeh(GridPlot)

In [ ]:
app = dashboards.view_features()
app.servable()

## add to template

In [ ]:
template = utils.get_template()
tmpl = pn.Template(template)
tmpl.add_panel('A',test1)
tmpl.add_panel('B',main)
tmpl.servable()

## widget/plot jslink

In [9]:
import bokeh.models.widgets as bhw
from bokeh.models.callbacks import CustomJS

loc = pnw.TextInput(name='location',value='',width=180)
#s = bhw.Slider(name='loc',start=0,end=100,value=2)
s=pnw.IntSlider(name='value',start=0,end=100,value=20)

p = plotters.test2(plot_width=800)
pp = pn.panel(p)

def callback(target, event):
    print (event)
    #target.object = target.x_range.start
    
pp.link(s, callbacks={'object': callback}) 
#pp.jslink(s, **{'x_range.start': 'value'})

jsupdateplot = '''
    r = target.x_range.end - target.x_range.start
    target.x_range.start = source.value
    target.x_range.end = source.value+r
'''
s.jslink(p, code={'value': jsupdateplot}) #value='x_range.start')

jsupdateloc = '''   
    if (source.start>0){
       target.value = Math.round(source.start)+':'+Math.round(source.end)
    }
'''
#pp.jslink(loc, code={'x_range.start': jsupdateloc})

#p.js_on_event('tap', callback)
#p.x_range.js_on_change('start', callback)

pn.Column(loc,s, pp)

Column
    [0] TextInput(name='location', width=180)
    [1] IntSlider(end=100, name='value', value=20)
    [2] Bokeh(Figure)

In [73]:
markdown = pn.pane.Markdown("<b>Markdown display</b>", width=400)
text_input = pn.widgets.TextInput(value="Markdown display")

code = '''
    target.text = '<b>' + source.value + '</b>'
'''
link = text_input.jslink(markdown, code={'value': code})

pn.Row(text_input, markdown)

Row
    [0] TextInput(value='Markdown display')
    [1] Markdown(str, width=400)

In [ ]:
p = figure(x_range=(0, 100), y_range=(0, 10),plot_height=200,tools="xpan,xwheel_zoom")
p.rect(50,5,10,4,color='red')
p.rect(80,5,10,4,color='blue')

## vcf plot

In [ ]:
import vcf
vcf_reader = vcf.Reader(open('wt_mbovis.vcf', 'r'))
for record in vcf_reader:
    print (record)

In [3]:
df = utils.vcf_to_dataframe('wt_mbovis.vcf')
df

,chrom,var_type,sub_type,start,end,REF,ALT,QUAL,DP,AO,RO
0,NC_002945.4,snp,ts,1175360,1175361,C,[T],3366.8700,168,138,30
1,NC_002945.4,snp,ts,1383347,1383348,C,[T],2882.2100,146,120,25
2,NC_002945.4,indel,ins,2258856,2258861,GCACG,[GCCACG],3702.5400,136,133,0
3,NC_002945.4,snp,ts,2299329,2299330,G,[A],5217.1700,247,210,37
4,NC_002945.4,indel,ins,2773171,2773192,TCGCCGTCGCGGCCCGTCGTT,[TCGCCGCCGTCGCGGCCCGTCGTT],2371.8700,134,99,0
5,NC_002945.4,indel,del,2776134,2776143,GCCCGACGC,[CCCCC],39.9753,26,4,2
6,NC_002945.4,indel,ins,3209292,3209309,ACCCCCCCGAGCCACAT,[ACCCCCCCCGAGCCACAT],134.1400,15,11,4
7,NC_002945.4,snp,tv,3549600,3549601,C,[A],64.0267,22,14,8
8,NC_002945.4,indel,ins,3830706,3830723,GGGCGCCCCCCGAGGGT,[GGGCGGCGCCCCCCGAGGGT],1631.9600,75,67,0


In [9]:
reload(plotters)
reload(utils)
p = plotters.plot_vcf('wt_mbovis.vcf',start=2772150,end=2774150)
pn.pane.Bokeh(p)

Bokeh(Figure)

In [59]:
from bokeh.io import curdoc
from bokeh.themes import Theme
curdoc().theme = Theme(json={'attrs': {
# apply defaults to Figure properties
'Figure': {
    'toolbar_location': None,
    'outline_line_color': None,
    'min_border_right': 10,
    'sizing_mode': 'stretch_width'
},
'Grid': {
    'grid_line_color': None,
},
'Title': {
    'text_font_size': '14pt'
},
# apply defaults to Axis properties
'Axis': {
    'minor_tick_out': None,
    'minor_tick_in': None,
    'major_label_text_font_size': '11pt',
    'axis_label_text_font_size': '13pt',
}

}})

In [10]:
from bokeh.plotting import figure, output_file, show
from bokeh.themes import built_in_themes

p=plotters.dummy_plot()
show(p)